In [9]:
import numpy as np
import pandas as pd
import os
import itertools

import networkx as nx
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [10]:
#Instruments dataset
#import data
user = os.environ["USERNAME"]

#from home
filename = "instrumentsdf_simNames_with_nstats.pkl"
datafolder = "C:/Users/{}/Dropbox/University/MscDataScience_Birkbeck/thesis_project/data/".format(user)

inst = pd.read_pickle(datafolder+filename)
inst.head().transpose()

uid,2744:79/231,2861:79/232,2932:79/233,1472:489/688,2042:512/645
customer_id,2004008,2004008,2004008,2004009,2004009
customer_name_1,Martinique Ltd,Martinique Ltd,Martinique Ltd,Sydney LLC,Sydney LLC
debtor_id,79,79,79,489,512
debtor_name_1,First Probabilists LLC,First Probabilists LLC,First Probabilists LLC,Newman Loggings Ltd,Mollie Nubia Energy Co.
invoice_number,2744,2861,2932,1472,2042
invoice_date,2013-07-23 00:00:00,2013-07-30 00:00:00,2013-08-06 00:00:00,2013-08-13 00:00:00,2013-08-13 00:00:00
due_date,2013-08-02 00:00:00,2013-08-09 00:00:00,2013-08-16 00:00:00,2013-08-23 00:00:00,2013-08-23 00:00:00
invoice_amount,913.7,2233.45,1370.5,9195.1,4594.6
purchase_amount,0,0,0,0,0
purchase_amount_open,0,0,0,0,0


In [ ]:
#adding a column counting the 

In [7]:
for c in inst.columns:
    print(c)

customer_id
customer_name_1
debtor_id
debtor_name_1
invoice_number
invoice_date
due_date
invoice_amount
purchase_amount
purchase_amount_open
discharge_type
discharge_amount
discharge_date
posting_date
transaction_type
document_stack_id
booking_text
input_date
creation_date
factoring_type
debt_collection_date
last_payment_date
reminder_date
test_feature_tested
test_characteristic_id
bill_id
cancellation_date
value_date
currency
purchase_examination
prosecution
deduction_amount
payment_amount
payment_date
document_status
uid
ttype
nrecords
ttypeset
discharge_loss
has_impairment1
has_impairment2
has_impairment3
is_pastdue90
is_pastdue180
has_prosecution
last_payment_amount
total_repayment
total_impairment
is_open
we_payment_share
has_purchase
has_deduction
is_due
has_discharge
cash_check
unexpl
first_posting_date
last_posting_date
payment_date_mismatch
dd_due_date
dd_discharge_date
dd_input_date
dd_creation_date
dd_debt_collection_date
dd_last_payment_date
dd_reminder_date
dd_cancellation

In [8]:
#FLOWS
# - accumulated delay in payment (it needs to be considered before and after the node in order to establish what is the type of node / effective resistance of the node)

#EFFORT
# -count

In [36]:
def add_flow_stats(inst, igroup, idx, id, ii, prefix):
    """
    This function adds flow stats to each edge.
    inst: instruments dataframe sorted by invoice_date
    igroup: group of instruments between a certain buyer and a certain seller
    idx: instrument index in the igroup 
    id: instrument id
    ii: instrument features (literally the dataset sliced in correspondence of that instrument)
    """
    
    #adding counter of accumulated delay for each edge
    inst.loc[id, prefix+'acc_delay'] = sum(igroup.loc[(igroup['is_open']) & (igroup['is_due']), 'payment_date_mismatch'])
    
    #adding the same counter proportioned with the number of lendings for this couple
    inst.loc[id, prefix+'acc_delay_r'] = igroup.acc_delay/igroup.cd_lent_c
    
    #adding counter of accumulated delay for each edge considering closed instruments as well
    inst.loc[id, prefix+'hist_acc_delay'] = sum(igroup.loc[igroup['is_due'], 'payment_date_mismatch'])
    
    #adding the same counter proportioned with the number of lendings for this couple
    inst.loc[id, prefix+'hist_acc_delay_r'] = igroup.acc_delay/igroup.cd_lent_c
    
    #adding counter of previously lent in this customer/debtor pair (inst is sorted by invoice date)
    #inst.loc[id, prefix+"lent_c"] = idx 
    
    #adding counter of previously repaid instruments in this customer/debtor pair
    #to be repaid, the last payment date needs to be smaller than all the instrument date and the instrument needs to not be open
    #repaid = (igroup.loc[:, "last_payment_date"] < ii[decision_date_col]) & (~ igroup.loc[:, "is_open"]) #filter for repaid instruments in this customer/debtor pair
    #inst.loc[id, prefix+"repaid_c"] = sum(repaid) 
            
    #adding counter of previously impaired in this customer/debtor pair
    #inst.loc[id, prefix+"impaired1_c"] = sum(igroup.loc[repaid,"has_impairment1"])
    #inst.loc[id, prefix+"impaired2_c"] = sum(igroup.loc[repaid,"has_impairment2"])
        
    #counter of overdue in this customer/debtor pair (considering previous instruments)
    #previous = igroup.index[:idx] #previous instruments selector
    #inst.loc[id, prefix+"pastdue90_c"] = sum((igroup.loc[previous,"due_date"] < ii[decision_date_col] - datetime.timedelta(90)) & igroup.loc[previous,"is_pastdue90"])
    #inst.loc[id, prefix+"pastdue180_c"] = sum((igroup.loc[previous,"due_date"] < ii[decision_date_col] - datetime.timedelta(180)) & igroup.loc[previous,"is_pastdue180"])
        
    #adding trend in amount lent in this customer/debtor pair
    #inst.loc[id, prefix+"trend_a"] = 0 if idx<2 else series_trend(igroup.loc[previous,"invoice_amount"])
        
    #adding counter of weekend payments in this pair
    #inst.loc[id, prefix+"we_payment_share"] = igroup.loc[repaid, "we_payment_share"].agg("mean")
        
    #adding payment_date_mismatch stats
    #inst.loc[id, prefix+"pd_mismatch_mean"] = igroup.loc[repaid, "payment_date_mismatch"].agg("mean")
    #inst.loc[id, prefix+"pd_mismatch_std"] = igroup.loc[repaid, "payment_date_mismatch"].agg("std") 

In [26]:
#Adding flow stats attributes to the edges
prefix = "cd_" #stands for customer/debtor
g_cb = inst.groupby(["customer_name_1", "debtor_name_1"])
for (customer, debtor), igroup in g_cb:
    #for each instrument in this group, already sorted by invoice_date
    for idx, (id, ii) in enumerate(igroup.iterrows()):
        add_flow_stats(inst, igroup, idx, id, ii, prefix)

#Adding the ratio columns for the previously calculated stats
#cl = [prefix+"repaid_", prefix+"impaired1_", prefix+"impaired2_", 
#      prefix+"pastdue90_", prefix+"pastdue180_"]
#for c in cl:
#    inst[c+"r"] = inst[c+"c"] / inst[prefix+"lent_c"]

In [30]:
inst.cd_acc_delay.max()

1031.0

In [ ]:
def add_d_flow_stats(inst, igroup, idx, id, ii, prefix):
    """
    This function adds debtors/buyers flow stats.
    inst: instruments dataframe sorted by invoice_date
    igroup: group of instruments between a certain buyer and a certain seller
    idx: instrument index in the igroup 
    id: instrument id
    ii: instrument features (literally the dataset sliced in correspondence of that instrument)
    """
    
    #adding counter for number of edges for each debtor node
    inst.loc[id, prefix+'edges_number'] = len(igroup.customer_name_1.unique())
    
    #delay distribution
    inst.loc[id, prefix+'distrib_delay']= sum(igroup.acc_delay_r)/igroup.edges_number

In [ ]:
prefix='d_'
g_cb = inst.groupy(["debtor_name_1"])
for _, igroup in g_b:
    for idx, (id, ii) in enumerate(igroup.iterrows()):
        add_d_flow_stats(inst, igroup, idx, id, ii, prefix)
    